In [1]:
#import pandas, numpy, os, matplotlib, seaborn, scrpy libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
path = r'C:\Users\joelf\OneDrive\Documents\CareerFoundry\4 Python\Instacart Data'

In [2]:
customers = pd.read_csv(os.path.join(path, 'NewDataSets', 'customers.csv'))
ords_prods_merged = pd.read_pickle(os.path.join(path, 'NewDataSets' , 'orders_products_combined.pkl'))
customers.shape

(206209, 10)

In [3]:
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [4]:
#4
#based on brief we can drop: firstname, surnam. not needed for any of the questions posed. Also anonymity?
#need user_id for merge, gender, state, age, dependents, fam_status, income for marketing questions in brief
#rename STATE to state, Gender to gender, Age to age, n_dependants to num_dependants, family_status to marital_status
customers = customers.drop(columns = ['First Name', 'Surnam'])
customers.rename(columns = {'Gender':'gender', 'STATE':'state', 'Age':'age', 'n_dependants':'num_dpdnts', 'fam_status':'marital_status'}, inplace = True)
customers.head()

,user_id,gender,state,age,date_joined,num_dpdnts,marital_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
#5a
#check for mixed data types
for col in customers.columns.tolist():
    weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (customers[weird]) > 0:
        print (col)
#no mixed data types found

In [6]:
#5b
#find missing values by counting nulls
customers.isnull().sum()
#no nulls in data set


user_id           0
gender            0
state             0
age               0
date_joined       0
num_dpdnts        0
marital_status    0
income            0
dtype: int64

In [7]:
#5c
#check for d
#uplicate data
customersDuplicates = customers[customers.duplicated()]
customersDuplicates.shape
#no dupliacte values in data set

(0, 8)

In [8]:
#check data set after data wrangling
customers.head()

,user_id,gender,state,age,date_joined,num_dpdnts,marital_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [9]:
#6a
#preparing for merge
#checking user_id is same data type for merge
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   user_id         206209 non-null  int64 
 1   gender          206209 non-null  object
 2   state           206209 non-null  object
 3   age             206209 non-null  int64 
 4   date_joined     206209 non-null  object
 5   num_dpdnts      206209 non-null  int64 
 6   marital_status  206209 non-null  object
 7   income          206209 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 12.6+ MB


In [10]:
ords_prods_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 25 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   order_number            int64   
 3   orders_day_of_week      int64   
 4   order_time              int64   
 5   days_since_prior_order  float64 
 6   repeat_customer         int64   
 7   product_id              int64   
 8   add_to_cart_order       int64   
 9   reordered               int64   
 10  product_name            object  
 11  aisle_id                int64   
 12  department_id           int64   
 13  prices                  float64 
 14  _merge                  category
 15  price_label             object  
 16  busiest_day             object  
 17  two_busiest_days        object  
 18  busiest_hours           object  
 19  max_order               int64   
 20  loyalty_flag            object  
 21  avg_pr

User_id will be used to join the data sets

In [11]:
#drop _merge column to use again
ords_prods_merged = ords_prods_merged.drop(columns = '_merge')
ords_prods_merged.shape

(32404859, 24)

In [12]:
#6b
#merge the data sets using left join. Keep everything in ord_prods, and map customer information to each purchase
cust_ord_combined = ords_prods_merged.merge(customers, on = 'user_id', indicator= True)
cust_ord_combined['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [13]:
ords_prods_merged.shape

(32404859, 24)

In [14]:
customers.shape

(206209, 8)

In [15]:
cust_ord_combined.shape

(32404859, 32)

In [16]:
pd.options.display.max_columns = None
cust_ord_combined.head(100)
#looks right, size is right. 24 from ords_prods_merged, 8 from customers, minus 1 since joined on user_id, plus 1 for adding _merge

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,repeat_customer,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_label,busiest_day,two_busiest_days,busiest_hours,max_order,loyalty_flag,avg_price,spender_flag,med_order_freq,freq_flag,gender,state,age,date_joined,num_dpdnts,marital_status,income,_merge
0,2539329,1,1,2,8,NaN,0,196,1,0,Soda,77,7,9.0,mid_range product,normal day,normal day,average orders,10,New Customer,6.367797,Low Spender,20.5,Nonfrequent Customer,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,1,196,1,1,Soda,77,7,9.0,mid_range product,normal day,slowest day,average orders,10,New Customer,6.367797,Low Spender,20.5,Nonfrequent Customer,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,1,196,1,1,Soda,77,7,9.0,mid_range product,normal day,slowest day,average orders,10,New Customer,6.367797,Low Spender,20.5,Nonfrequent Customer,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,1,196,1,1,Soda,77,7,9.0,mid_range product,slowest day,slowest day,average orders,10,New Customer,6.367797,Low Spender,20.5,Nonfrequent Customer,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,1,196,1,1,Soda,77,7,9.0,mid_range product,slowest day,slowest day,average orders,10,New Customer,6.367797,Low Spender,20.5,Nonfrequent Customer,Female,Alabama,31,2/17/2019,3,married,40423,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3317979,15,5,4,15,17.0,1,14715,1,1,Coconut Water,98,7,4.0,low_range product,slowest day,slowest day,average orders,22,Regular Customer,3.980556,Low Spender,10.0,Frequent Customer,Female,Indiana,69,6/3/2019,0,divorced/widowed,54313,both
96,2685110,15,7,1,11,17.0,1,14715,3,1,Coconut Water,98,7,4.0,low_range product,normal day,busiest day,most orders,22,Regular Customer,3.980556,Low Spender,10.0,Frequent Customer,Female,Indiana,69,6/3/2019,0,divorced/widowed,54313,both
97,887727,15,9,2,13,7.0,1,14715,1,1,Coconut Water,98,7,4.0,low_range product,normal day,normal day,average orders,22,Regular Customer,3.980556,Low Spender,10.0,Frequent Customer,Female,Indiana,69,6/3/2019,0,divorced/widowed,54313,both
98,2600170,15,11,2,9,14.0,1,14715,1,1,Coconut Water,98,7,4.0,low_range product,normal day,normal day,average orders,22,Regular Customer,3.980556,Low Spender,10.0,Frequent Customer,Female,Indiana,69,6/3/2019,0,divorced/widowed,54313,both


In [17]:
#8 export
cust_ord_combined.to_pickle(os.path.join(path, 'NewDataSets', 'orders_products_customers_combined.pkl'))